In [1]:
input={
    'C_Type':['Company'],
	# 'S_Type':['Salaried'],
	'TRI':1,
	'Company_Name':'TREKONIC PRIVATE LIITED ',
	'City':'BANGALORE',
	'Area':['Bull Temple Road','Basavanagudi','Bengaluru Urban'],
	'State':'KARNATAKA',
	'Name':'SHOBHA H',
	'Sur_Name':'H',
	'Trust': False,
	'Pension':False,
 
	'Corporate':False,
	'Profession':'ENTREPRENAUR',
	'Exemption':False,
	'C_Name':'m',
	'GOVT.': False
}

In [60]:
input={
    'C_Type':['Company'],
	# 'S_Type':['Salaried'],
	'TRI':5,
	'Company_Name':'RAIMAXAL PHARAMATEUTICALS PRIVATE LIMITED',
	'City':'SURAT',
	'Area':['Ratanji Nagar','Mithi Khadi','Bhavani Nagar','Limbayat'],
	'State':'Gujarat',
	'Name':'RAJA SHEKAR',
	'Sur_Name':'S',
	'Trust': False,
	'Pension':False,
	'Corporate':False,
	'Profession':'ENTREPRENAUR',
	'Exemption':False,
	'C_Name':'r',
	'GOVT.': False
}

In [47]:
input={
    'C_Type':['Company'],
	# 'S_Type':['Salaried'],
	'TRI':5,
	'Company_Name':'HAXSPLOIT SECURITY SERVICES PRIVATE LIMITED',
	'City':'INDORE',
	'Area':['Kakad','Khandwa'],
	'State':'MADHYA PRADESH',
	'Name':'YASH SHENDGE',
	'Sur_Name':'S',
	'Trust': False,
	'Pension':False,
	'Corporate':False,
	'Profession':'ENTREPRENAUR',
	'Exemption':False,
	'C_Name':'h',
	'GOVT.': False
}

In [4]:
input={
    'C_Type':['Company'],
	# 'S_Type':['Salaried'],
	'TRI':5,
	'Company_Name':'jOURNEY ESCAPE',
	'City':'BANGALORE',
	'Area':['Marenahalli','Saraswathi Nagar'],
	'State':'KARNATAKA',
	'Name':'CHAMPA H N',
	'Sur_Name':'S',
	'Trust': False,
	'Pension':False,
	'Corporate':False,
	'Profession':'ENTREPRENAUR',
	'Exemption':False,
	'C_Name':'j',
	'GOVT.': False
}

In [2]:
default_params = {
    'C_Type': 'Def_C_Type',
    'S_Type': 'Def_S_Type',
    'C_Name': 'All',
    'Sur_Name': 'All',
    'Pension': False,
    'Trust': False,
    'Exemption': False,
    'TRI': 'All',
    'Area': 'All',
    'GOVT.':False,
    'NOT':None
}

In [3]:
import pandas as pd
import ast
lf=pd.read_excel('tan2.xlsx')

In [4]:

lf['C_Name1'] = lf['C_Name1'].apply(lambda x: '[' + x + ']')

In [5]:
lf['C_Name1'] = lf['C_Name1'].apply(ast.literal_eval)

In [6]:
import ast

# Define a function to convert the string to a dictionary
def convert_to_dict(data_string):
    return ast.literal_eval(data_string)

# Apply the conversion function to the 'ADDITIONAL DESCRIPTION' column
# df1['ADDITIONAL DESCRIPTION'] = df1['ADDITIONAL DESCRIPTION'].apply(convert_to_dict)
# data=convert_to_dict(df1['ADDITIONAL DESCRIPTION'][1])
import traceback

for i, row in lf.iterrows():
    try:
        lf.at[i, 'ADDITIONAL DESCRIPTION'] = convert_to_dict(row['ADDITIONAL DESCRIPTION'])
    except Exception as e:
        print(f"Error occurred at row {i+1}:")
        print(row)
        traceback.print_exc()

In [7]:
filtered_df=lf[lf['CITY']== input['City']]

In [8]:
dropped_count = 0

def filter_data1(input_dict, key, row, default_param=None):
    global dropped_count  # Use the global dropped_count variable
    remaining_data = row['ADDITIONAL DESCRIPTION']['REMAINING_DATA']
    filtered_data = [
        data for data in remaining_data
        if (data.get(key) == input_dict.get(key) or data == input_dict)
        or (default_param is not None and data.get(key) == default_param.get(key))
    ]

    if filtered_data:
        row['ADDITIONAL DESCRIPTION']['REMAINING_DATA'] = filtered_data
        # print(filtered_data)
        return row
    else:
        dropped_count += 1  # Increment the dropped_count variable
        return None

filtered_df=filtered_df.apply(lambda row: filter_data1(input, 'C_Type', row, default_params), axis=1).dropna()


In [9]:
fd=filtered_df.copy()

In [10]:
fs=fd.apply(lambda row: filter_data1(input, 'Pension', row, default_params), axis=1).dropna()
# fs = pd.concat(fs.tolist())

In [11]:
fs=fs.apply(lambda row: filter_data1(input, 'Govt.', row, default_params), axis=1).dropna()
# fs = pd.concat(fs.tolist())

In [12]:
dropped_count = 0

def filter_data(input_dict, key, row, default_param=None):
    global dropped_count  # Use the global dropped_count variable
    c_namedata = row['C_Name1']
    remaining_data = row['ADDITIONAL DESCRIPTION']['REMAINING_DATA']
    filtered_data = []

    for data_list in c_namedata:
        for data in data_list:
            if input_dict.get(key).lower().startswith(data):
                filtered_data.append(data_list)

    if filtered_data:
        row['ADDITIONAL DESCRIPTION']['REMAINING_DATA'] = remaining_data
        return pd.DataFrame(row).transpose()
    
    else:
            dropped_count += 1  # Increment the dropped_count variable
            return None

fss = fs.apply(lambda row: filter_data(input, 'Company_Name', row, default_params), axis=1).dropna()

if fss.empty:
    # default_row = {'C_NAME1': [], 'ADDITIONAL DESCRIPTION': {'REMAINING_DATA': default_params}}
    fss = fs.copy()
else:
    fss = pd.concat(fss.tolist())


In [13]:
dropped_count = 0

def filter_data(input_dict, key, row, default_param=None):
    global dropped_count  # Use the global dropped_count variable
    remaining_data = row['ADDITIONAL DESCRIPTION']['REMAINING_DATA']
    filtered_data = []
    
    for data in remaining_data:
        area_list = data.get(key)
        
        for area in area_list:
            if any(input_area in area for input_area in input_dict[key] for area in area_list):
                filtered_data.append(data)
                break

    if filtered_data:
        row['ADDITIONAL DESCRIPTION']['REMAINING_DATA'] = filtered_data
        return pd.DataFrame(row).transpose()
    # elif default_param is not None:
    #     row['ADDITIONAL DESCRIPTION']['REMAINING_DATA'] = default_param
    #     return pd.DataFrame(row).transpose()
    else:
        dropped_count += 1
        return None

fz = fss.apply(lambda row: filter_data(input, 'Area', row, default_params), axis=1).dropna()
if fz.empty:
    # default_row = {'C_NAME1': [], 'ADDITIONAL DESCRIPTION': {'REMAINING_DATA': default_params}}
    fz = fss.copy()
else:
    fz = pd.concat(fz.tolist())

In [14]:
fz

,Unnamed: 0,SR. NO.,RCC,AREA CODE,AO TYPE,RANGE CODE,AO NO.,DESCRIPTION,ADDITIONAL DESCRIPTION,CITY,C_Name,C_Name1
39,39,40,BLR,KAR,WT,182,2,"TDS WARD 2(2), BANGALORE","{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",BANGALORE,"['""R"" TO ""Z""']","[[r, s, t, u, v, w, x, y, z]]"


In [15]:
fd

,Unnamed: 0,SR. NO.,RCC,AREA CODE,AO TYPE,RANGE CODE,AO NO.,DESCRIPTION,ADDITIONAL DESCRIPTION,CITY,C_Name,C_Name1
31,31,32,BLR,KAR,CT,181,1,"TDS CIRCLE-1(1), BANGALORE","{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",BANGALORE,"['""A"" TO ""F"",\'0\' to \'4\' ']","[[a, b, c, d, e, f, 0, 1, 2, 3, 4]]"
32,32,33,BLR,KAR,WT,181,1,"TDS WARD-(1)(1), BANGALORE","{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",BANGALORE,"['""A"" TO ""F"",\'0\' to \'4\' ']","[[a, b, c, d, e, f, 0, 1, 2, 3, 4]]"
33,33,34,BLR,KAR,WT,181,2,"TDS WARD-(1) (2), BANGALORE","{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",BANGALORE,"['""G"" to ""L"" ']","[[g, h, i, j, k, l]]"
34,34,35,BLR,KAR,WT,181,3,"TDS WARD-(1)(3), BANGALORE","{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",BANGALORE,['All'],[[All]]
35,35,36,BLR,KAR,CT,181,4,"ACIT TDS(OSD), BANGALORE","{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",BANGALORE,['All'],[[All]]
37,37,38,BLR,KAR,CT,182,1,"TDS CIRCLE 2(1), BANGALOR","{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",BANGALORE,"['""M"" TO ""Q"",\'5\' to \'9\' ']","[[m, n, o, p, q, 5, 6, 7, 8, 9]]"
38,38,39,BLR,KAR,WT,182,1,"TDS WARD 2(1), BANGALORE","{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",BANGALORE,"['""M"" TO ""Q"",\'5\' to \'9\' ']","[[m, n, o, p, q, 5, 6, 7, 8, 9]]"
39,39,40,BLR,KAR,WT,182,2,"TDS WARD 2(2), BANGALORE","{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",BANGALORE,"['""R"" TO ""Z""']","[[r, s, t, u, v, w, x, y, z]]"
40,40,41,BLR,KAR,WT,182,3,"TDS WARD 2(3), BANGALORE","{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",BANGALORE,['All'],[[All]]
52,52,53,BLR,KAR,CT,185,1,"TDS CIRCLE-3(1), BANGALORE","{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",BANGALORE,['All'],[[All]]
